In [1]:
import import_ipynb
from keras.layers import Input
from pretrained_model import pretrained_resnet
from rpn import rpn
from classifier import classifier
from keras.utils import generic_utils
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
import xml.etree.ElementTree as ET
import numpy as np
import os
import random
import pprint
import Datagenerators as D
from Dataextract import data
import Finaltruelabels as F
import losses
from DataAugment import data_augment
from keras import backend as K
import math
#import all the ipynb files here

Using TensorFlow backend.


importing Jupyter notebook from pretrained_model.ipynb
importing Jupyter notebook from rpn.ipynb
importing Jupyter notebook from classifier.ipynb
importing Jupyter notebook from Datagenerators.ipynb
importing Jupyter notebook from DataAugment.ipynb
importing Jupyter notebook from Dataextract.ipynb
importing Jupyter notebook from Finaltruelabels.ipynb
importing Jupyter notebook from losses.ipynb


In [2]:
num_rois = 32

In [3]:
all_images, classes_count, class_mapping = data(r'C:\Users\ANSHUL\Downloads\VOCdevkit')#path(simple_label_path)

if 'bg' not in classes_count:
    classes_count['bg'] = 0
    class_mapping['bg'] = len(class_mapping)

In [4]:
print('Training images per class:')
pprint.pprint(classes_count)
print('Number of classes (including bg) = {}'.format(len(classes_count)))
random.shuffle(all_images)
num_imgs = len(all_images)
train_imgs = [s for s in all_images if s['imageset'] == 'trainval']
print(class_mapping)
#val_imgs = [s for s in all_images if s['imageset'] == 'test']  Validation data is not used 

Training images per class:
{'aeroplane': 1002,
 'bg': 0,
 'bicycle': 837,
 'bird': 1271,
 'boat': 1059,
 'bottle': 1561,
 'bus': 685,
 'car': 2492,
 'cat': 1277,
 'chair': 3056,
 'cow': 771,
 'diningtable': 800,
 'dog': 1598,
 'horse': 803,
 'motorbike': 801,
 'person': 17401,
 'pottedplant': 1202,
 'sheep': 1084,
 'sofa': 841,
 'train': 704,
 'tvmonitor': 893}
Number of classes (including bg) = 21
{'person': 0, 'aeroplane': 1, 'tvmonitor': 2, 'train': 3, 'boat': 4, 'dog': 5, 'chair': 6, 'bird': 7, 'bicycle': 8, 'bottle': 9, 'sheep': 10, 'diningtable': 11, 'horse': 12, 'motorbike': 13, 'sofa': 14, 'cow': 15, 'car': 16, 'cat': 17, 'bus': 18, 'pottedplant': 19, 'bg': 20}


In [5]:
def get_img_output_length(width, height):
    def get_output_length(input_length):
        # zero_pad
        input_length += 6
        # apply 4 strided convolutions
        filter_sizes = [7, 3, 1, 1]
        stride = 2
        for filter_size in filter_sizes:
            input_length = (input_length - filter_size + stride) // stride
        return input_length

    return get_output_length(width), get_output_length(height)

In [5]:
def base_layer(inp):
    #inp = Input(shape = (None,None,3))
    base_network = pretrained_resnet(inp)
    base_out = base_network.layers[-1].output
    return base_out

In [6]:
i_inp = Input(shape = (None,None,3))
roi_inp = Input(shape = (None,4))

In [7]:
s_layers = base_layer(i_inp)

In [8]:
#data_gen_train = D.anch_rpn(train_imgs, classes_count, get_img_output_length, mode='train')
data_gen_train = D.anch_rpn(train_imgs, classes_count,mode='train')
##data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, nn.get_img_output_length, mode='val')
#Since we do not have any validation data at this point

In [9]:
def train_frcnn(anchor_num=9):
    inp = Input(shape=(None, None, 3))
    proposals = Input(shape=(None,3))

    base_network = pretrained_resnet(inp)
    base_out = base_network.layers[-1].output
    
    rpn_class_out,rpn_regr_out= rpn(base_out, anchor_num)[0],rpn(base_out, anchor_num)[1]
    
    #classifier_img = classifier(base_out, proposals, 32,num_classes=21)
    
    return [rpn_class_out,rpn_regr_out,base_out]

In [10]:
from keras.layers import Convolution2D
def rpn(base_layers, num_anchors):
    x = Convolution2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(
        base_layers)

    x_class = Convolution2D(num_anchors, (1, 1), activation='sigmoid', kernel_initializer='uniform',
                            name='rpn_out_class')(x)
    x_regr = Convolution2D(num_anchors * 4, (1, 1), activation='linear', kernel_initializer='zero',
                           name='rpn_out_regress')(x)

    return [x_class, x_regr, base_layers]


In [11]:
# anchor box scales
anchor_box_scales = [128, 256, 512]
# anchor box ratios
anchor_box_ratios = [[1, 1], [1, 2], [2, 1]]

In [12]:
num_anchors = len(anchor_box_scales) * len(anchor_box_ratios)
#rpn = train_frcnn(num_anchors)
# rpn will carry [x_class, x_regr, base_layers]
rpn = rpn(s_layers, num_anchors)

In [13]:
classifier = classifier(s_layers, roi_inp, num_rois, num_classes=len(classes_count))
# This will return classes and regr after the roi pooling

In [14]:
print(s_layers)

Tensor("bn5c_branch2b/cond/Merge:0", shape=(?, ?, ?, 512), dtype=float32)


In [15]:
model_rpn = Model(i_inp, rpn[:2])
model_classifier = Model([i_inp, roi_inp], classifier)

In [16]:
model_all = Model([i_inp, roi_inp], rpn[:2] + classifier)

In [17]:
# try:
#        print('loading weights from {}'.format(base_net_weights))
#        model_rpn.load_weights(model_path, by_name=True)
#        model_classifier.load_weights(model_path, by_name=True)
#    except Exception as e:
#        print(e)
#        print('Could not load pretrained model weights. Weights can be found in the keras application folder '
#              'https://github.com/fchollet/keras/tree/master/keras/applications)
## This snippet load the weights of rpn and classifier
## We have to change the location path 

In [18]:
opt = Adam(lr=1e-5)
opt_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=opt,
                      loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=opt_classifier,
                         loss=[losses.class_loss_cls,losses.class_loss_regr(len(classes_count) - 1)],
                         metrics=['accuracy'])
model_all.compile(optimizer='sgd', 
                  loss='mae')


In [19]:
epoch_length = 1000
num_epochs = 3000
i = 0

In [20]:
losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
best_loss = np.inf

In [22]:
for epoch_num in range(num_epochs):
    progbar = generic_utils.Progbar(epoch_length)
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))
    while True:
            if len(rpn_accuracy_rpn_monitor) == epoch_length and True:
                mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor)) / len(rpn_accuracy_rpn_monitor)
                rpn_accuracy_rpn_monitor = []
                print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(
                            mean_overlapping_bboxes, epoch_length))
                if mean_overlapping_bboxes == 0:
                    print('RPN is not producing bounding boxes that overlap the ground truth boxes.' 
                              'Check RPN settings or keep training.')
            X, Y, img_data = next(data_gen_train)
            #print(np.asarray(Y).shape)
            loss_rpn = model_rpn.train_on_batch(X, Y)

            P_rpn = model_rpn.predict_on_batch(X)

            result = F.apply_rpn(P_rpn[0], P_rpn[1],overlap_thresh=0.7,max_boxes=300)
            #print(result.shape)
            X2, Y1, Y2, IouS = F.last_iou(result, img_data, class_mapping)
            #print(X2.shape)
            #print(Y1.shape)
            #print(Y2.shape)
            if X2 is None:
                rpn_accuracy_rpn_monitor.append(0)
                rpn_accuracy_for_epoch.append(0)
                continue

                
            neg_samples = np.where(Y1[0, :, -1] == 1)
            pos_samples = np.where(Y1[0, :, -1] == 0)

            if len(neg_samples) > 0:
                neg_samples = neg_samples[0]
            else:
                neg_samples = []

            if len(pos_samples) > 0:
                pos_samples = pos_samples[0]
            else:
                pos_samples = []
                
                
            rpn_accuracy_rpn_monitor.append(len(pos_samples))
            rpn_accuracy_for_epoch.append((len(pos_samples)))
               
            if num_rois > 1:
                if len(pos_samples) < num_rois // 2 and len(pos_samples)!=0:
                    selected_pos_samples = pos_samples.tolist()
                else:
                    if(len(pos_samples)!=0):
                        selected_pos_samples = np.random.choice(pos_samples, num_rois // 2, replace=False).tolist()
                    else:
                        selected_pos_samples=[]
                        
                try:
                    selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples),
                                                                replace=False).tolist()
                except:
                    if(len(neg_samples)!=0):
                        selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples),
                                                                replace=True).tolist()
                    else:
                        selected_neg_samples=[]

                samples = selected_pos_samples + selected_neg_samples
            else:
                    # in the extreme case where num_rois = 1, we pick a random pos or neg sample
                selected_pos_samples = pos_samples.tolist()
                selected_neg_samples = neg_samples.tolist()
                if np.random.randint(0, 2):
                    samples = random.choice(neg_samples)
                else:
                    samples = random.choice(pos_samples)

            loss_class = model_classifier.train_on_batch([X, X2[:, samples, :]],
                                                             [Y1[:, samples, :], Y2[:, samples, :]])
            losses[i, 0] = loss_rpn[1]
            losses[i, 1] = loss_rpn[2]
            losses[i, 2] = loss_class[1]
            losses[i, 3] = loss_class[2]
            losses[i, 4] = loss_class[3]
                
            i += 1
                
            progbar.update(i,[('rpn_cls', np.mean(losses[:i, 0])), ('rpn_regr', np.mean(losses[:i, 1])),
                                ('detector_cls', np.mean(losses[:i, 2])),
                                ('detector_regr', np.mean(losses[:i, 3]))])
                
            if i == epoch_length:
                loss_rpn_cls = np.mean(losses[:, 0])
                loss_rpn_regr = np.mean(losses[:, 1])
                loss_class_cls = np.mean(losses[:, 2])
                loss_class_regr = np.mean(losses[:, 3])
                class_acc = np.mean(losses[:, 4])
                mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
                rpn_accuracy_for_epoch = []                
                    
                if True:
                    print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(
                            mean_overlapping_bboxes))
                    print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                    print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                    print('Loss RPN regression: {}'.format(loss_rpn_regr))
                    print('Loss Detector classifier: {}'.format(loss_class_cls))
                    print('Loss Detector regression: {}'.format(loss_class_regr))  
                        
                        
                curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
                i = 0
                
                if curr_loss < best_loss:
                    best_loss = curr_loss
                    model_all.save_weights(model_path)
                
                break

                # save model
                model_all.save_weights(model_path)
print('Training complete, exiting.')


Epoch 1/3000
  15/1000 [..............................] - ETA: 20:49:04 - rpn_cls: 8.3311 - rpn_regr: 177.8219 - detector_cls: 2.4211 - detector_regr: 9.6715

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

